# Chapter04: LangChain Foundation

Export OPENAI_API_KEY

In [1]:
from dotenv import load_dotenv

loaded = load_dotenv()

## 4.1 Try a Simple Q&A

In [3]:
from langchain_openai import OpenAI

model = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
output = model.invoke("Please introduce yourself.")
print(output)



Hello, my name is [Name]. I am [age] years old and I am from [city/country]. I am a [occupation/study] and I am passionate about [interest/hobby]. In my free time, I enjoy [activity/hobby]. I am excited to get to know you and learn more about you.


## 4.2 Multi-turn Dialogue

`SystemMessage`, `HumanMessage`, and `AIMessage` corresponds to components in OpenAI's Chat Completions.

---

```text
`SystemMessage` <--> `"role": "system"`

`HumanMessage` <--> `"role": "user"`

`AIMessage` <--> `"role": "assistant"`
```


---

```json
{
    "model": "gpt-4o"
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hi! I am Chen!"},
        {"role": "assistant", "content": "Hi, Chen. How can I help you?"},
        {"role": "user", "content": "Do you know my name?"},
    ],
    // …omitted… //
}
```

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", temperature=0)

# `content=` can be omitted
messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("Hi! I am Chen!"),
    AIMessage(content="Hi, Chen. How can I help you?"),
    HumanMessage(content="Do you know my name?"),
]

ai_message = model.invoke(messages)
print(ai_message.content)

Yes, you mentioned that your name is Chen. How can I assist you today?


## 4.3 Streaming Mode

It is similar to use `streaming mode` with to use based on OpenAI's Chat Completions.

In [6]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", temperature=0)

messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("Hi! Please say something in 50 words."),
]

for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)

Hello! I hope you're having a wonderful day. Remember to take a moment to appreciate the little things around you. Whether it's a warm cup of coffee, a good book, or a friendly smile, these small joys can make a big difference. Stay positive and keep moving forward!

## 4.4 Prompt Template

In [8]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""Please come up with a recipe for the following dish.
                                      
Dish name: {dish}""")

prompt_value = prompt.invoke({"dish": "Curry"})
print(prompt_value.text)

Please come up with a recipe for the following dish.
                                      
Dish name: Curry


```text
If we want to use OpenAI's Chat Completions format, we can use `ChatPromptTemplate`.
```

In [10]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        # {"role": "system", "content": "Please come up with a recipe for the dish provided by users."}
        ("system", "Please come up with a recipe for the dish provided by users."),
        # {"role": "user", "content": "{dish}"}
        ("human", "{dish}"),
    ]
)

prompt_value = prompt.invoke({"dish": "Curry"})
print(prompt_value)

messages=[SystemMessage(content='Please come up with a recipe for the dish provided by users.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Curry', additional_kwargs={}, response_metadata={})]


### MessagePlaceholder

In [16]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
    ]
)

prompt_value = prompt.invoke(
    {
        "chat_history": [
            HumanMessage(content="Hi! I am Chen!"),
            AIMessage("Hi! Chen! How can I help you?"),
        ],
        "input": "Do you know my name?"
    }
)

print(prompt_value)

messages=[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hi! I am Chen!', additional_kwargs={}, response_metadata={}), AIMessage(content='Hi! Chen! How can I help you?', additional_kwargs={}, response_metadata={}), HumanMessage(content='Do you know my name?', additional_kwargs={}, response_metadata={})]


### LangSmith

We can use prompts created by others by using `langsmith`.

In [14]:
from langsmith import Client

client = Client()
prompt = client.pull_prompt("oshima/recipe")

prompt_value = prompt.invoke({"dish": "カレー"})
print(prompt_value)

/opt/anaconda3/envs/ai-agent-py310/lib/python3.10/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


messages=[SystemMessage(content='ユーザーが入力した料理のレシピを考えてください。', additional_kwargs={}, response_metadata={}), HumanMessage(content='カレー', additional_kwargs={}, response_metadata={})]


### Multimodal Model's Input Template 

In [19]:
import base64

def jpg_to_data_uri(path: str) -> str:
    '''
    This function is used to convert *.jpg to urls which can be feed into GPT-4o.
    '''
    with open(path, "rb") as f:
        b64 = base64.b64encode(f.read()).decode("ascii")
    return f"data:image/jpeg;base64,{b64}"

image_path = "example_image.jpeg"
image_uri = jpg_to_data_uri(image_path)

In [21]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "user",
            [
                {"type": "text", "text": "Please describe this image."},
                {"type": "image_url", "image_url": {"url": "{image_url}"}}
            ],
        ),
    ]
)

prompt_value = prompt.invoke({"image_url": image_uri})

In [22]:
model = ChatOpenAI(model="gpt-4o", temperature=0)
ai_message = model.invoke(prompt_value)
print(ai_message.content)

The image shows a simple, empty room with wooden flooring. There is a large window with sheer white curtains allowing light to enter. An air conditioning unit is mounted above the window. On the floor, there is a suitcase and several bags, including a tote bag and a backpack, placed against the wall. The room appears to be minimally furnished and tidy.


## 4.5 PydanticOutputParser

We can get Python objects from LLMs' outputs by using `PydanticOutputParser`.

In [2]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

class Recipe(BaseModel):
    ingredientt: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")

output_parser = PydanticOutputParser(pydantic_object=Recipe)

Here are the instructions fed to the LLMs to guide their outputs. These instructions will **be embedded into** the prompts.

In [3]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredientt": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredientt", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredientt", "steps"]}
```


In [10]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate(
    [
        (
            "system",
            "Please come up with a recipe based on the given dish name.\n\n{format_instructions}",
        ),
        ("human", "{dish}"),
    ]
)

prompt_with_format_instructions = prompt.partial(
    format_instructions=format_instructions
)


In [11]:
prompt_value = prompt_with_format_instructions.invoke({"dish": "Curry"})
print("=== role: system ===")
print(prompt_value.messages[0].content)
print("=== role: user ===")
print(prompt_value.messages[1].content)

=== role: system ===
Please come up with a recipe based on the given dish name.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredientt": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredientt", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredientt", "steps"]}
```
=== role: user ===
Curry


In [14]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", temperature=0)

ai_message = model.invoke(prompt_value)
print(ai_message.content)

```json
{
    "ingredientt": [
        "2 tablespoons vegetable oil",
        "1 large onion, finely chopped",
        "3 cloves garlic, minced",
        "1 tablespoon fresh ginger, grated",
        "2 teaspoons ground cumin",
        "2 teaspoons ground coriander",
        "1 teaspoon turmeric",
        "1 teaspoon chili powder",
        "1 teaspoon garam masala",
        "1 can (14 oz) diced tomatoes",
        "1 cup coconut milk",
        "1 pound chicken breast, cut into bite-sized pieces",
        "Salt to taste",
        "Fresh cilantro, chopped (for garnish)"
    ],
    "steps": [
        "Heat the vegetable oil in a large pan over medium heat.",
        "Add the chopped onion and sauté until it becomes translucent.",
        "Stir in the minced garlic and grated ginger, cooking for another minute until fragrant.",
        "Add the ground cumin, coriander, turmeric, chili powder, and garam masala. Stir well to coat the onions and release the spices' aroma.",
        "Pour in the

Now, we sucessfully obtain the Python object `Recipe`.

- A string specifying the output format is automatically generated from the `Recipe` class definition.

- The LLM’s output could then be easily converted into an instance of the `Recipe` class.

**However, if the LLM returns incomplete JSON text, an error will be raised.**

In [15]:
recipe = output_parser.invoke(ai_message)
print(type(recipe))
print(recipe)

<class '__main__.Recipe'>
ingredientt=['2 tablespoons vegetable oil', '1 large onion, finely chopped', '3 cloves garlic, minced', '1 tablespoon fresh ginger, grated', '2 teaspoons ground cumin', '2 teaspoons ground coriander', '1 teaspoon turmeric', '1 teaspoon chili powder', '1 teaspoon garam masala', '1 can (14 oz) diced tomatoes', '1 cup coconut milk', '1 pound chicken breast, cut into bite-sized pieces', 'Salt to taste', 'Fresh cilantro, chopped (for garnish)'] steps=['Heat the vegetable oil in a large pan over medium heat.', 'Add the chopped onion and sauté until it becomes translucent.', 'Stir in the minced garlic and grated ginger, cooking for another minute until fragrant.', "Add the ground cumin, coriander, turmeric, chili powder, and garam masala. Stir well to coat the onions and release the spices' aroma.", 'Pour in the diced tomatoes and coconut milk, stirring to combine.', 'Add the chicken pieces to the pan, ensuring they are submerged in the sauce.', 'Season with salt to 

## 4.6 StrOutputParser

In [19]:
from langchain_core.messages import AIMessage
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

ai_message = AIMessage(content="Hi! I am an AI assistant.")
print(type(ai_message))
print(ai_message.content)
print("-----------------------------")
output = output_parser.invoke(ai_message)
print(type(output))
print(output)

<class 'langchain_core.messages.ai.AIMessage'>
Hi! I am an AI assistant.
-----------------------------
<class 'str'>
Hi! I am an AI assistant.


## 4.7 LangChain Expression Language (LCEL)

### Try a simple example

In [21]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate(
    [
        ("system", "Please come up with a recipe based on the given dish name."),
        ("human", "{dish}"),
    ]
)

model = ChatOpenAI(model_name="gpt-4o", temperature=0)

**Create a `Chain`**

In [22]:
chain = prompt | model

In [23]:
ai_message = chain.invoke({"dish": "Curry"})
print(ai_message.content)

Certainly! Here's a simple and delicious recipe for a classic Chicken Curry:

### Ingredients:
- 1.5 lbs (about 700g) chicken thighs, boneless and skinless, cut into bite-sized pieces
- 2 tablespoons vegetable oil
- 1 large onion, finely chopped
- 3 cloves garlic, minced
- 1 tablespoon fresh ginger, grated
- 2 teaspoons ground cumin
- 2 teaspoons ground coriander
- 1 teaspoon turmeric powder
- 1 teaspoon garam masala
- 1 teaspoon chili powder (adjust to taste)
- 1 can (14 oz) coconut milk
- 1 cup chicken broth
- 2 tablespoons tomato paste
- Salt and pepper to taste
- Fresh cilantro, chopped, for garnish
- Cooked basmati rice, for serving

### Instructions:

1. **Prepare the Chicken:**
   - Season the chicken pieces with salt and pepper.

2. **Cook the Aromatics:**
   - Heat the vegetable oil in a large pan over medium heat.
   - Add the chopped onion and sauté until it becomes translucent, about 5 minutes.
   - Stir in the garlic and ginger, cooking for another 2 minutes until fragrant

### From `prompt template`  to  `parser`

1. Prepare the `Recipe` class and `parser`

In [25]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

class Recipe(BaseModel):
    ingredientt: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")

output_parser = PydanticOutputParser(pydantic_object=Recipe)

2. Prepare the `prompt template` and the `model`

In [26]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate(
    [
        ("system", "Please come up with a recipe based on the given dish name.\n\n{format_instructions}"),
        ("human", "{dish}"),
    ]
)

prompt_with_format_instructions = prompt.partial(
    format_instructions=output_parser.get_format_instructions()
)

# Guide the model to use `JSON mode`
model = ChatOpenAI(model="gpt-4o", temperature=0).bind(
    response_format={"type": "json_object"}
)

3. Create a chain

In [27]:
chain = prompt_with_format_instructions | model | output_parser

4. Obtain the results

In [28]:
recipe = chain.invoke({"dish": "Curry"})
print(type(recipe))
print(recipe)

<class '__main__.Recipe'>
ingredientt=['2 tablespoons vegetable oil', '1 large onion, finely chopped', '3 cloves garlic, minced', '1 tablespoon fresh ginger, grated', '2 tablespoons curry powder', '1 teaspoon ground cumin', '1 teaspoon ground coriander', '1/2 teaspoon turmeric', '1/2 teaspoon cayenne pepper', '1 can (14 oz) coconut milk', '1 can (14 oz) diced tomatoes', '1 pound chicken breast, cut into bite-sized pieces', 'Salt to taste', 'Fresh cilantro, chopped (for garnish)'] steps=['Heat the vegetable oil in a large pan over medium heat.', 'Add the chopped onion and sauté until translucent, about 5 minutes.', 'Stir in the minced garlic and grated ginger, cooking for another 2 minutes.', "Add the curry powder, ground cumin, ground coriander, turmeric, and cayenne pepper. Stir well to coat the onions and cook for 1 minute to release the spices' aroma.", 'Pour in the coconut milk and diced tomatoes, stirring to combine.', 'Add the chicken pieces to the pan, ensuring they are submerge

**A simpler example with using `.with_structured_output`**

However, some models may not support this function.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Please come up with a recipe based on the given dish name."),
        ("human", "{dish}"),
    ]
)

model = ChatOpenAI(model="gpt-4o")

# `.with_structured_output is uesd to guide the model to output in JSON format.`
chain = prompt | model.with_structured_output(Recipe)

recipe = chain.invoke({"dish": "Curry"})
print(type(recipe))
print(recipe)

<class '__main__.Recipe'>
ingredients=['2 tablespoons vegetable oil', '1 large onion, finely chopped', '3 cloves garlic, minced', '1 tablespoon fresh ginger, grated', '2 tablespoons curry powder', '1 teaspoon ground cumin', '1 teaspoon ground coriander', '1/2 teaspoon turmeric', '1/4 teaspoon cayenne pepper (optional)', '1 can (14 oz) coconut milk', '1 can (14 oz) diced tomatoes', '1 pound chicken breast, cut into bite-sized pieces', 'Salt and pepper to taste', '2 cups cooked rice', 'Fresh cilantro, chopped (for garnish)'] steps=['Heat the vegetable oil in a large pot over medium heat.', 'Add the chopped onion and sauté until translucent, about 5 minutes.', 'Stir in the minced garlic and grated ginger, cooking until fragrant, about 1 minute.', 'Add the curry powder, ground cumin, ground coriander, turmeric, and cayenne pepper (if using) to the pot. Stir well to coat the onions and garlic with the spices.', 'Pour in the coconut milk and diced tomatoes. Stir to combine.', 'Add the chicke

## 4.8 Attemption of RAG

In [5]:
from langchain_community.document_loaders import GitLoader

def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")

loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

# type of `raw_docs`: list
raw_docs = loader.load()
print(len(raw_docs))

419


Split documents into chunks (whose size is 1000) based on number of characters

Iterate through each document, splitting it into chunks of size 1000 with an overlap of 50 between consecutive chunks to maintain continuity.

In [6]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

docs = text_splitter.split_documents(raw_docs)
print(len(docs))

Created a chunk of size 6803, which is longer than the specified 1000
Created a chunk of size 3302, which is longer than the specified 1000
Created a chunk of size 1851, which is longer than the specified 1000
Created a chunk of size 1639, which is longer than the specified 1000
Created a chunk of size 9269, which is longer than the specified 1000
Created a chunk of size 2579, which is longer than the specified 1000
Created a chunk of size 17715, which is longer than the specified 1000
Created a chunk of size 1700, which is longer than the specified 1000
Created a chunk of size 1135, which is longer than the specified 1000
Created a chunk of size 1126, which is longer than the specified 1000
Created a chunk of size 1098, which is longer than the specified 1000
Created a chunk of size 1433, which is longer than the specified 1000
Created a chunk of size 1300, which is longer than the specified 1000
Created a chunk of size 1166, which is longer than the specified 1000
Created a chunk of 

1469


`CharacterTextSplitter` doesn’t blindly cut at a fixed length (like 1000); it splits on the separators we specify (by default **["\n\n", "\n", " "]**) and then builds chunks no larger than `chunk_size` at those split points. (We can enable a `force split` option to strictly split every 1000 characters.)

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

query = "Is there a document loader for loading data from AWS S3?"

vector = embeddings.embed_query(query)
print(len(vector))
print(vector)

1536
[0.02095509134232998, -0.005143425893038511, 0.03000243566930294, -0.02733272686600685, 0.049665048718452454, 0.024027373641729355, -0.01388883963227272, 0.0217390526086092, 0.02608262561261654, -0.021198755130171776, 0.007648926228284836, -0.00954526662826538, -0.01538260467350483, -0.006626597139984369, -0.011483983136713505, 0.06420012563467026, 0.0356808640062809, -0.0003671511076390743, -0.04525791108608246, 0.027290351688861847, 0.031485605984926224, 0.037524234503507614, -0.039070967584848404, 0.020912714302539825, 0.01807350106537342, -0.015859337523579597, -0.022607767954468727, 0.033519670367240906, -0.007018578238785267, -0.10068614035844803, -0.010276257991790771, -0.05674189329147339, -0.03449432551860809, 0.0468682125210762, -0.02498084120452404, 0.034642644226551056, 0.023052718490362167, 0.015107158571481705, -0.0075959558598697186, -0.03667670488357544, -0.0009289684239774942, -0.021654300391674042, 0.02124113216996193, 0.01549914013594389, 0.007272836286574602, 0

In [11]:
from langchain_chroma import Chroma

db = Chroma.from_documents(docs, embeddings)

retriever = db.as_retriever()

query = "Is there a document loader for loading data from AWS S3?"

context_docs = retriever.invoke(query)
print(f"len = {len(context_docs)}")

first_doc = context_docs[0]
print(f"metadata = {first_doc.metadata}")
print(first_doc.page_content)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


len = 4
metadata = {'file_name': 'aws.mdx', 'file_path': 'docs/docs/integrations/providers/aws.mdx', 'file_type': '.mdx', 'source': 'docs/docs/integrations/providers/aws.mdx'}
## Document loaders

### AWS S3 Directory and File

>[Amazon Simple Storage Service (Amazon S3)](https://docs.aws.amazon.com/AmazonS3/latest/userguide/using-folders.html)
> is an object storage service.
>[AWS S3 Directory](https://docs.aws.amazon.com/AmazonS3/latest/userguide/using-folders.html)
>[AWS S3 Buckets](https://docs.aws.amazon.com/AmazonS3/latest/userguide/UsingBucket.html)

See a [usage example for S3DirectoryLoader](/docs/integrations/document_loaders/aws_s3_directory).

See a [usage example for S3FileLoader](/docs/integrations/document_loaders/aws_s3_file).

```python
from langchain_community.document_loaders import S3DirectoryLoader, S3FileLoader
```

### Amazon Textract

>[Amazon Textract](https://docs.aws.amazon.com/managedservices/latest/userguide/textract.html) is a machine 
> learning (ML) serv

In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_template('''\
Please refer to the following context and answer the question.

Context: """
{context}             
"""

Question: {question}
''')

model = ChatOpenAI(model="gpt-4o", temperature=0)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

output = chain.invoke(query)
print(output)

Yes, there are document loaders for loading data from AWS S3. The context mentions two specific loaders: `S3DirectoryLoader` and `S3FileLoader`. These loaders are used to load data from AWS S3 directories and files, respectively.


## 4.9 The Basic Components of LCEL: `Runnable` & `RunnableSequence`

Define the default variables

In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Please come up with a recipe based on the given dish name."),
        ("human", "{dish}"),
    ]
)

model = ChatOpenAI(model="gpt-4o", temperature=0)

output_parser = StrOutputParser()

Obtain `output` without using `chain`

In [ ]:
prompt_value = prompt.invoke({"dish": "Curry"})
ai_message = model.invoke(prompt_value)
output = output_parser.invoke(ai_message)
print(output)

Certainly! Here's a simple and delicious recipe for a classic Chicken Curry:

### Ingredients:
- 1.5 lbs (about 700g) chicken thighs, boneless and skinless, cut into bite-sized pieces
- 2 tablespoons vegetable oil
- 1 large onion, finely chopped
- 3 cloves garlic, minced
- 1 tablespoon fresh ginger, grated
- 2 teaspoons ground cumin
- 2 teaspoons ground coriander
- 1 teaspoon turmeric powder
- 1 teaspoon garam masala
- 1 teaspoon chili powder (adjust to taste)
- 1 can (14 oz) coconut milk
- 1 cup chicken broth
- 2 tablespoons tomato paste
- Salt and pepper to taste
- Fresh cilantro, chopped, for garnish
- Cooked basmati rice, for serving

### Instructions:

1. **Prepare the Chicken:**
   - Season the chicken pieces with salt and pepper.

2. **Cook the Aromatics:**
   - Heat the vegetable oil in a large pan over medium heat.
   - Add the chopped onion and sauté until it becomes translucent, about 5 minutes.
   - Stir in the garlic and ginger, cooking for another 2 minutes until fragrant

We can obtain the same results by using `RunnableSequence`.

```text
When we invoke a `RunnableSequence, it invokes each of the linked Runnable objects in sequence.`
```

In [ ]:
chain = prompt | model | output_parser

output = chain.invoke({"dish": "Curry"})
print(output)

Certainly! Here's a simple and delicious recipe for a classic Chicken Curry:

### Ingredients:
- 1.5 lbs (about 700g) chicken breast or thighs, cut into bite-sized pieces
- 2 tablespoons vegetable oil
- 1 large onion, finely chopped
- 3 cloves garlic, minced
- 1 tablespoon fresh ginger, grated
- 2 teaspoons ground cumin
- 2 teaspoons ground coriander
- 1 teaspoon turmeric powder
- 1 teaspoon garam masala
- 1 teaspoon chili powder (adjust to taste)
- 1 can (14 oz) coconut milk
- 1 cup chicken broth
- 2 tablespoons tomato paste
- Salt and pepper to taste
- Fresh cilantro, chopped, for garnish
- Cooked basmati rice, for serving

### Instructions:

1. **Prepare the Chicken:**
   - Season the chicken pieces with salt and pepper.

2. **Cook the Aromatics:**
   - Heat the vegetable oil in a large pan over medium heat.
   - Add the chopped onion and sauté until it becomes translucent, about 5 minutes.
   - Stir in the garlic and ginger, cooking for another 2 minutes until fragrant.

3. **Add t

`Runnable` objects' executable methods: `invoke`, `stream`, `batch`

**stream method**

To execute a Runnable in **streaming mode**, use the `stream` method as follows.

In [19]:
chain = prompt | model | output_parser

for chunk in chain.stream({"dish": "Curry"}):
    print(chunk, end="", flush=True)

Certainly! Here's a simple and delicious recipe for a classic Chicken Curry:

### Ingredients:
- 1.5 lbs (about 700g) chicken thighs or breasts, cut into bite-sized pieces
- 2 tablespoons vegetable oil
- 1 large onion, finely chopped
- 3 cloves garlic, minced
- 1 tablespoon fresh ginger, grated
- 2 teaspoons ground cumin
- 2 teaspoons ground coriander
- 1 teaspoon turmeric powder
- 1 teaspoon garam masala
- 1 teaspoon chili powder (adjust to taste)
- 1 can (14 oz) coconut milk
- 1 cup chicken broth
- 2 tablespoons tomato paste
- Salt and pepper to taste
- Fresh cilantro, chopped, for garnish
- Cooked basmati rice, for serving

### Instructions:

1. **Prepare the Chicken:**
   - Season the chicken pieces with salt and pepper.

2. **Cook the Aromatics:**
   - Heat the vegetable oil in a large pan over medium heat.
   - Add the chopped onion and sauté until it becomes translucent, about 5 minutes.
   - Stir in the garlic and ginger, cooking for another 2 minutes until fragrant.

3. **Add 

**batch method**

By using the **batch** method, you can process `multiple inputs` together.

In [20]:
chain = prompt | model | output_parser

outputs = chain.batch([{"dish": "Curry"}, {"dish": "udon"}])
print(outputs)

["Certainly! Here's a simple and delicious recipe for a classic Chicken Curry:\n\n### Ingredients:\n- 1.5 lbs (about 700g) chicken thighs or breasts, cut into bite-sized pieces\n- 2 tablespoons vegetable oil\n- 1 large onion, finely chopped\n- 3 cloves garlic, minced\n- 1 tablespoon fresh ginger, grated\n- 2 teaspoons ground cumin\n- 2 teaspoons ground coriander\n- 1 teaspoon turmeric powder\n- 1 teaspoon garam masala\n- 1 teaspoon chili powder (adjust to taste)\n- 1 can (14 oz) diced tomatoes\n- 1 cup coconut milk\n- Salt to taste\n- Fresh cilantro, chopped, for garnish\n\n### Instructions:\n\n1. **Prepare the Ingredients:**\n   - Chop the onion, mince the garlic, and grate the ginger. Cut the chicken into bite-sized pieces.\n\n2. **Cook the Aromatics:**\n   - Heat the vegetable oil in a large pan over medium heat. Add the chopped onion and sauté until it becomes translucent, about 5 minutes.\n   - Add the minced garlic and grated ginger, and cook for another 2 minutes until fragrant.

```text
In this way, any class inheriting from `Runnable` can be invoked through a unified interface using methods like invoke, stream, and batch. Furthermore, asynchronous counterparts —- ainvoke, astream, and abatch —- are also provided.
```

### 4.10 Multiple chains

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")
output_parser = StrOutputParser()

In [3]:
cot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Please think step by step to answer this question."),
        ("human", "{question}"),
    ]
)

cot_chain = cot_prompt | model | output_parser

In [10]:
summarize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Please just give me the final answer from the step-by-step solution."),
        # Only one placeholder is required.
        # After running the `cot_chain``, it produces a dictionary like {"text": "…"}.”
        ("human", "{text}"),
    ]
)

summarize_chain = summarize_prompt | model | output_parser

In [11]:
cot_summarize_chain = cot_chain | summarize_chain

cot_summarize_chain.invoke({"question": "10 + 2 * 3"})

'16'